In [8]:
from utils.preprocess import process_dataset
from utils.env_setup import get_stock_env
from environments.MutliAgentStockTradingEnv import MultiAgentStockTradingEnv
train, valid = process_dataset()

Successfully added technical indicators
[*********************100%%**********************]  1 of 1 completed
Shape of DataFrame:  (3310, 8)
Successfully added vix
Successfully added turbulence index


/home/abyaadrafid/anaconda3/envs/RLAC/lib/python3.10/site-packages/finrl/meta/preprocessor/preprocessors.py:106: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method="ffill").fillna(method="bfill")
/home/abyaadrafid/Desktop/TUM/thesis/utils/preprocess.py:36: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  processed_full = processed_full.fillna(method="bfill")


In [62]:
env = get_stock_env(train)
marlenv = MultiAgentStockTradingEnv(env)

In [63]:
from stable_baselines3 import A2C
agent1 = A2C("MlpPolicy", env, verbose=1)
agent2 = A2C("MlpPolicy", env, verbose=1)

Using cpu device
Using cpu device


In [64]:
actions1, states = agent1.predict(env.reset())
actions2, states = agent2.predict(env.reset())

In [75]:
import numpy as np
actions=np.stack(([actions1,actions2]),axis=0)

In [77]:
actions[0].shape

(1, 29)

In [80]:
agent_id = [0,1]
states, rewards, dones, _ = marlenv.step(actions, agent_id)

In [34]:
random.shuffle(agent_id)

In [35]:
agent_id

[1, 0]

In [81]:
rewards

[array([0.01182447], dtype=float32), array([-0.32305568], dtype=float32)]

In [89]:
arr=np.random.uniform(20,200,(5,2))

In [108]:
c = np.random.uniform(0,2000,(5,1))


In [109]:
np.hstack((arr,c))

array([[ 106.64553536,   95.52270585,  477.27293256],
       [ 122.96375307,  154.4810241 , 1407.87793144],
       [ 159.787166  ,   22.58863945,  191.58486151],
       [ 145.63735317,  101.14944826, 1782.7126752 ],
       [  69.46164658,   50.50295043, 1506.63722016]])

In [114]:
import matplotlib.pyplot as plt
demand = np.zeros(100 - 50)
consumer_prices = np.random.uniform(100, 50, 20)
# Counting consumers that will buy tickets at possible prices
i = 0
for price in range(50, 100):

    demand[i] = sum(price <= bid for bid in consumer_prices)
    i += 1

# Plotting the demand curve
plt.plot(demand, range(50, 100))
plt.xlabel("Q")
plt.ylabel("P")
plt.title("Demand for Airline Tickets")
plt.show()

/tmp/ipykernel_32402/1556221523.py:16: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [115]:
demand

array([20., 19., 18., 18., 18., 18., 18., 18., 18., 17., 17., 16., 16.,
       14., 13., 13., 12., 12., 12., 12., 12., 12., 12., 12., 12., 12.,
       12., 12., 12., 12., 12., 12., 12., 12., 11., 11.,  9.,  8.,  8.,
        8.,  5.,  5.,  4.,  3.,  3.,  3.,  3.,  1.,  0.,  0.])